In [1]:
import bs4
import json
import requests
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
def getTrans(link):
    page = requests.get(link)
    soup = bs4.BeautifulSoup(page.text)
    return soup.find('div',{'id':'transcript'}).findAll(text = True)
    

In [4]:
def findnth(haystack, needle, n):
    parts= haystack.split(needle, n+1)
    if len(parts)<=n+1:
        return -1
    return len(haystack)-len(parts[-1])-len(needle)

In [5]:
def createJSON (link, text):
    lnk = 'http://millercenter.org' + link
    pres_id = link[findnth(link,'/',1)+1 : findnth(link,'/',2)]
    speech_id = link[findnth(link,'/',3)+1 : len(link)]
    ps = {}
    ps['transcript'] = getTrans(lnk)
    ps['link'] = lnk
    ps['title'] = text[0:text.find("(")]
    ps['pres_id'] = pres_id
    ps['es_id'] = pres_id +'_'+ speech_id
    ps['date'] = text[text.find("(")+1:text.find(")")]
    return ps


In [ ]:
page = requests.get('http://millercenter.org/president/speeches')
soup = bs4.BeautifulSoup(page.text)
links = soup.select('div.title a[href^=/president]')
#print links
for link in links:
    ps = createJSON(link.attrs['href'],link.text)
    if ps['pres_id'] == 'obama':
        es.index(index='obama', doc_type='speeches', id=ps['es_id'], body = ps)

In [3]:
res = es.count(index='obama', doc_type='speeches')
print(json.dumps(res,indent = 2))

{
  "count": 7, 
  "_shards": {
    "successful": 5, 
    "failed": 0, 
    "total": 5
  }
}
